In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

C:\Users\NE-A-U\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\NE-A-U\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\NE-A-U\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\NE-A-U\Anaconda3\envs\tensorflow-g

In [100]:
janken_array = np.array([[1,0,0],[0,1,0],[0,0,1]])

In [101]:
janken_class = ['グー','チョキ','パー']

In [102]:
#過去何回分の手を覚えているか
n = 3

In [103]:
Jprev = np.zeros(3*n*2)

In [104]:
#過去の手をランダムに初期化
for i in range(2*n):
    j = np.random.randint(0,3)
    Jprev[3*i:3*i+3] = janken_array[j]

In [105]:
#現在の手をランダムに初期化
j = janken_array[np.random.randint(0,3)]

In [106]:
Jprev_set = np.array([Jprev])
Jnow_set = np.array([j])

In [107]:
def decord_janken(arr):
    
    for i,k in enumerate(janken_array):
        if all(arr == k):
            return i

In [125]:
def create_model():
    #モデルの初期化
    model = keras.Sequential()
    
    model.add(keras.layers.Dense(units=50,
                                input_dim=Jprev_set.shape[1],
                                kernel_initializer='glorot_uniform',
                                bias_initializer='zeros',
                                activation='tanh'))
    
    model.add(keras.layers.Dense(units=50,
                                input_dim=50,
                                kernel_initializer='glorot_uniform',
                                bias_initializer='zeros',
                                activation='tanh'))
    
    model.add(keras.layers.Dense(units=3,
                                input_dim=50,
                                kernel_initializer='glorot_uniform',
                                bias_initializer='zeros',
                                activation='softmax'))
    
    optimizer = keras.optimizers.Adam()
    
    model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy')
    
    
    return model

In [126]:
model = create_model()

In [127]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 50)                950       
_________________________________________________________________
dense_15 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_16 (Dense)             (None, 3)                 153       
Total params: 3,653
Trainable params: 3,653
Non-trainable params: 0
_________________________________________________________________


In [113]:
history = model.fit(Jprev_set,Jnow_set)

1/1 [==============================] - 0s 153ms/sample - loss: 1.0554


In [ ]:

    print("1:グー、2:チョキ、3:パー")
    
    win = 0
    draw = 0
    lose = 0
    
    try:
        while True:
            try:
                j = int(input())-1
                
            except(SyntaxError,NameError,ValueError):
                continue
                
            if j<0 or j>2:
                continue
            
            Jprev_set = np.array([Jprev])
            Jnow_set = np.array([j])
            
            #コンピュータが過去の手から人間の現在の手を予測
            jpredict = model.predict_classes(Jprev_set)
            print(jpredict)
            
            #人間の手を代入
            your_choice = j
            
            #予測を元にコンピュータが決めた手
            comp_choice = (jpredict[0]+2)%3
            
            #お互いの手の表示
            print("あなた："+janken_class[your_choice]+", コンピュータ："+janken_class[comp_choice])
            
            #スコアを記録
            if your_choice == comp_choice:
                draw += 1
            elif your_choice == (comp_choice+1)%3:
                lose += 1
            else:
                win += 1
                
            #勝敗の結果表示
            print("あなたの勝ち: {0}, 負け：{1}, あいこ： {2}".format(win,lose,draw))
            
            #過去の手（入力データ）と現在の手（ターゲット）とでオンライン学習
            history = model.fit(Jprev_set,Jnow_set)
            
            #過去の手の末尾に現在のコンピュータの手を追加
            Jprev = np.append(Jprev[3:], janken_array[comp_choice])
            
            #過去の手の末尾に現在の人間の手を追加
            Jprev = np.append(Jprev[3:], janken_array[comp_choice])
            
    except KeyboardInterrupt:
        pass
            

1:グー、2:チョキ、3:パー
1
[1]
あなた：グー, コンピュータ：グー
あなたの勝ち: 0, 負け：0, あいこ： 1
1/1 [==============================] - 0s 130ms/sample - loss: 1.6513
2
[1]
あなた：チョキ, コンピュータ：グー
あなたの勝ち: 0, 負け：1, あいこ： 1
1/1 [==============================] - 0s 3ms/sample - loss: 0.8336
13
3
[0]
あなた：パー, コンピュータ：パー
あなたの勝ち: 0, 負け：1, あいこ： 2
1/1 [==============================] - 0s 3ms/sample - loss: 1.1369
3
[2]
あなた：パー, コンピュータ：チョキ
あなたの勝ち: 0, 負け：2, あいこ： 2
1/1 [==============================] - 0s 3ms/sample - loss: 0.6748
3
[2]
あなた：パー, コンピュータ：チョキ
あなたの勝ち: 0, 負け：3, あいこ： 2
1/1 [==============================] - 0s 2ms/sample - loss: 0.7542
3
[2]
あなた：パー, コンピュータ：チョキ
あなたの勝ち: 0, 負け：4, あいこ： 2
1/1 [==============================] - 0s 2ms/sample - loss: 0.8886
1
[1]
あなた：グー, コンピュータ：グー
あなたの勝ち: 0, 負け：4, あいこ： 3
1/1 [==============================] - 0s 2ms/sample - loss: 1.4517
3
[1]
あなた：パー, コンピュータ：グー
あなたの勝ち: 1, 負け：4, あいこ： 3
1/1 [==============================] - 0s 3ms/sample - loss: 1.9218
2
[1]
あなた：チョキ, コンピュータ：グー
あなたの勝ち: 1, 負け：5, あいこ： 